---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2022</h1>
<h1 style="font-size: 250%;">Problem Set #4</h1>
<h1>Issued Thursday, 3/10/2022; Due Thursday, 3/24/2022</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Feel free to get help from others, but **the work you submit in should be your own.** Furthermore, this assignment may end up overlapping Problem Set #5, so plan accordingly. 

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [2]:
# Replace the following string values with the requested information
class Student:
    first = "Owen"
    last = "McCadden"
    onyen = "owenmc"
    pid = "730314080"

The following problems make use of the following Insulin protein sequences

In [3]:
human = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"
pig = "MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN"

In [4]:
from numpy import *

def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):  # test for a match ("diagonal street")
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

In [5]:
def LCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    elif (b[i,j] == 3):
        return LCS(b,v,i-1,j-1) + v[i-1]
    elif (b[i,j] == 2):
        return LCS(b,v,i,j-1)
    else:
        return LCS(b,v,i-1,j)

In [6]:
def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result
    

In [7]:
import numpy

def GlobalAlign(v, w, indel, match, mismatch, adjacent):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(0,len(v)+1):
        for j in range(0,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = s[i-1,j] + indel
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = s[i,j-1] + indel
                b[i,j] = 2
                continue
            if abs(ord(v[i-1]) - ord(w[j-1])) == 0: # check if there is a match
                score = s[i-1,j-1] + match
            elif abs(ord(v[i-1]) - ord(w[j-1])) == 1: # check if adjacent
                score = s[i-1,j-1] + adjacent
            else: # check for all other mismatches
                score = s[i-1,j-1] + mismatch
                
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            else:
                b[i,j] = 3
    return (s, b)

---
**Problem #1:** Find the longest common subsequence (LCS) shared between human and pig insulin, its length, and the alignment that it implies.

In [8]:
v = human
w = pig
s, b = findLCS(v,w)

lcs = LCS(b, v, b.shape[0]-1,b.shape[1]-1)
print(lcs, len(lcs))

align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print("v =", align[0])
print("w =", align[1])

MALWRLLPLLALLALWPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKRREAEQGLQLALEGQKRGIVEQCCTSICSLYQLENYCN 87
v = MALWM_RLLPLLALLALWGPD_PA_A_AFVNQHLCGSHLVEALYLVCGERGFFYTPKT_RREAEDL__Q_GS__________LQP_LALEGSL__QKRGIVEQCCTSICSLYQLENYCN
w = MALW_TRLLPLLALLALW___APAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPK_ARREAE__NPQAG_AVELGGGLGGLQ_ALALEG__PPQKRGIVEQCCTSICSLYQLENYCN


---
**Problem #2:** Find an optimal *global alignment* between human and pig insulin assuming the following scoring from the "Alignment Game" described in Lecture 15, slide 4:

* +1 for a match
* 0 for a mismatch
* -1 for an indel

What is score for this alignment and the alignment itself? 

In [9]:
v = human
w = pig

s, b = GlobalAlign(v, w, -1, 1, 0, 0)
print(s)
lcs = LCS(b, v, b.shape[0]-1,b.shape[1]-1)
print(lcs, len(lcs))

print(s[-1, -1])

align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print("v =", align[0])
print("w =", align[1])

[[   0   -1   -2 ... -106 -107 -108]
 [  -1    1    0 ... -104 -105 -106]
 [  -2    0    2 ... -102 -103 -104]
 ...
 [ -96  -94  -92 ...   75   74   73]
 [ -97  -95  -93 ...   74   76   75]
 [ -98  -96  -94 ...   73   75   77]]
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN 98
77
v = MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQ_GS__L_______QPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
w = MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN


**Problem #3:** According to the rules of the "Alignment Game", what score does your alignment from **Problem 1** achieve? How would the rules of the alignment game need to be changed so that your solution for **Problem 1** would be the optimal answer?

In [71]:
v = human
w = pig

s, b = GlobalAlign(v, w, -1, 1, -206, -206)
lcs = LCS(b, v, b.shape[0]-1,b.shape[1]-1)
print(lcs, len(lcs))

print(s[-1, -1])

align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print("v =", align[0])
print("w =", align[1])

MALWRLLPLLALLALWPPAAFVNQHLCGSHLVEALYLVCGERGFFYTPKRREAEQGLQLALEGQKRGIVEQCCTSICSLYQLENYCN 87
55
v = MALW_MRLLPLLALLALW_GP_DPA_AAFVNQHLCGSHLVEALYLVCGERGFFYTPK_TRREAE__DLQ_G___SL_______Q_PLALEG__SLQKRGIVEQCCTSICSLYQLENYCN
w = MALWT_RLLPLLALLALWA_PA_PAQA_FVNQHLCGSHLVEALYLVCGERGFFYTPKA_RREAENP__QAGAVE_LGGGLGGLQA_LALEGPP__QKRGIVEQCCTSICSLYQLENYCN


In [22]:
print(len(v) + len(w))

206


---
**Problem #4:** Find the length of the longest common subsequence of the human insulin sequence with the same sequence reversed.

In [72]:
v = human
w = human[::-1]

s, b = findLCS(v,w)
lcs = LCS(b, v, b.shape[0]-1,b.shape[1]-1)

print(lcs, len(lcs))

LLLLALPQLSLEAERTKTREAELSLQPLALLLL 33


**Problem #5:** Find an optimal *global alignment* between the human insulin sequence and the human insulin sequence reversed using the following scoring:

* +1 for a match
* 0 if the peptides are adjacent alphabetically (i.e. 'M' is adjacent to both 'L' and 'N')
* -1 for all other mismatches
* -1 for an indel

Provide the alignment's score and the alignment itself.

Find an alternate optimal alignment for these sequences that uses the same scoring function but is constrained to match the substring 'EAE', which appears exactly once in each sequence and give its score.

In [73]:
v = human
w = human[::-1]

s, b = GlobalAlign(v, w, -1, 1, -1, 0)
lcs = LCS(b, v, b.shape[0]-1, b.shape[1]-1)

print(lcs, len(lcs))

print(s[-1,-1])

align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)

print("v = {}".format(align[0]))
print("w = {}".format(align[1]))

MALMRLPLLALLALWGPDPAFVNQLCGSLALYLVGRGFFYTPTRREAEGSLQLALGSLQKRGIVEQCCTSICSLYQLENYCN 82
-49
v = MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALY_LVCGERG_F_FY_T_PKTRREA_EDLQGS_LQPLA__LEGS__L_QKRGIVEQCCTSICSLY__QLE_NYC_N
w = NCY_NEL_QYLSCISTCCQEVI__GRKQ_LSGE_L__ALPQLS_G_QLDEAERRTKP_TYFFGRE___GCVLY_LAEVL_HSGCLHQNVFAAAPDPGWLALLALLPLLRMWLAM


In [74]:
v_start = v.find("EAE")
w_start = w.find("EAE")

first = GlobalAlign(v[:v_start], w[:w_start], -1, 1, -1, 0)[0][-1,-1]
second = GlobalAlign(v[v_start:], w[w_start:], -1, 1, -1, 0)[0][-1,-1]
score = first + second
print(score)

-53


---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.